In [9]:
import numpy as np
import time
import os

import keras

# import matplotlib.pyplot as plt
import pandas as pd
import random

import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
# from sklearn.utils import shuffle
from sklearn.utils import class_weight
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_absolute_error
from tensorflow.keras.optimizers import Adam
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = 'false'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tensorboard = TensorBoard(log_dir="./logs")

DEBUG = True

In [10]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']

data_directory = '/home/joren/Coding/cryptodata/Normalized_labelled/'
max_df_length = 25000

#####################################
frame_size = 120
layers = 5
layer_sizes = [512]*layers
dropouts = [0.1]*layers
batchnormalizations = [0]*layers
learning_rate = 0.0001
optimizer = Adam(learning_rate)

class_weight = {0: 1.,
                1: 50.,
                2: 50.}
#####################################

In [11]:
field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data)

In [12]:
def random_file():
    filenames = []
    for file in os.listdir(data_directory):
        filename = os.fsdecode(file)
        filenames.append(filename)
        
    randomname = filenames[random.randint(0, len(filenames)-1)]
    if randomname.endswith(".bin"): 
        print(f"reading file: {os.path.join(data_directory, randomname)}")
        return os.path.join(data_directory, randomname)

In [13]:
class DQN(tf.keras.Model):
    def __init__(self, n_actions, feature_size, layers = 2, layer_sizes = [128, 128], dropouts = [0.1, 0], batchnormalizations = [0, 0], optimizer='adam'):
        super().__init__()
        self._n_actions = n_actions
        self._feature_size = feature_size
        self._frame_size = frame_size

        self._model = self.create_model(layers, layer_sizes, dropouts, batchnormalizations, optimizer)
    
    def create_model(self, layers, layer_sizes, dropouts, batchnormalizations, optimizer):
        model = Sequential()

        for i in range(0, layers):
            if i == 0:
                model.add(LSTM(units=layer_sizes[i], return_sequences = True, input_shape = (self._frame_size, self._feature_size)))
            elif i == layers:
                model.add(LSTM(units=layer_sizes[i]))
            elif i >= len(layer_sizes):
                model.add(LSTM(units=layer_sizes[0], return_sequences = True))
            else:
                model.add(LSTM(units=layer_sizes[i], return_sequences = True))


            if i < len(dropouts) and dropouts[i] > 0:
                model.add(Dropout(dropouts[i]))
            if i < len(batchnormalizations) and batchnormalizations[i] == 1:
                model.add(BatchNormalization()) 
        
        model.add(Dense(units=self._n_actions, activation='softmax'))
        
        model.compile(optimizer=optimizer, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

        return model


In [18]:
# train-test split
def df_split(df):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, Y, test_size=0.1, shuffle=False)

    y_train_raw = to_categorical(y_train_raw, 3).tolist()
    y_test_raw = to_categorical(y_test_raw, 3).tolist()

    X_train = []
    y_train = []
    # y_train_weights = []
    for i in range(frame_size, X_train_raw.shape[0]): #frame size up to size of array
        X_train.append(X_train_raw[i-frame_size:i])
        y_train.append(y_train_raw[i-frame_size:i]) # dit wil ik dus graag veranderen naar y_train_raw[i] zodat we enkel op het einde de output hebben
    X_train, y_train = np.array(X_train), np.array(y_train)

    X_test = []
    y_test = []
    for i in range(frame_size, X_test_raw.shape[0]): #frame size up to size of array
        X_test.append(X_test_raw[i-frame_size:i])
        y_test.append(y_test_raw[i-frame_size:i])
    X_test, y_test = np.array(X_test), np.array(y_test)

    if DEBUG:
        print(f"""
        X_train shape: {X_train.shape}
        y_train shape: {y_train.shape}
        """)

        print(f"X_train[0]: {X_train[0]}")
        print(f"y_train[0]: {y_train[0]}")

    return X_train, X_test, y_train, y_test

In [19]:
# model initialization
dqn = DQN(3, 17, layers, layer_sizes, dropouts, batchnormalizations, optimizer)

def test_accuracy(model):
    y_pred_raw = np.array(model.predict(X_test))
    y_pred = np.argmax(y_pred_raw, axis=-1, keepdims=True)
    y_pred = y_pred.flatten()
    y_test_2 = np.argmax(y_test, axis=-1, keepdims=True)
    y_test_2 = y_test_2.flatten()
    print(f"""
    Class. report:
    {classification_report(y_test_2, y_pred)}
    """)

    cf = confusion_matrix(y_test_2, y_pred)

    print(cf)
    print(accuracy_score(y_test_2, y_pred) * 100) 


In [21]:
for i in range(100):
    file = random_file()
    data = read_bin_full_file(file)

    df = pd.DataFrame(data, columns=column_names) # variable from cell 1

    df.fillna(0, inplace=True)
    # if df.isnull().values.any():
    #     print('nan values found')
    #     continue

    df = df.iloc[240:]

    if len(df) > max_df_length:
        randstart = random.randint(0, len(df)-max_df_length)
        df = df.iloc[randstart:randstart+max_df_length]

    X_train, X_test, y_train, y_test = df_split(df)

    print(X_train.shape)
    print(y_train.shape)

    dqn._model.fit(X_train, y_train, epochs = 3, batch_size = 256, callbacks=[tensorboard])
    test_accuracy(dqn._model)

reading file: /home/joren/Coding/cryptodata/Normalized_labelled/EOSBULLUSDT.bin

        X_train shape: (22380, 120, 17)
        y_train shape: (22380, 120, 3)
        
X_train[0]: [[-0.00766234 -0.00722115 -0.0070922  ...  0.          0.0077215
   0.00759063]
 [-0.00503941  0.         -0.00334147 ...  0.          0.00714286
   0.00714286]
 [-0.00206319  0.00375356  0.00335268 ...  0.          0.00542706
   0.00206745]
 ...
 [-0.00688776 -0.00409207 -0.0026704  ...  0.          0.00732083
   0.00025687]
 [ 0.0003828  -0.01622846 -0.01069317 ...  0.          0.00652758
  -0.00255102]
 [-0.01408982  0.00277532 -0.00012579 ...  0.          0.01442062
   0.01429118]]
y_train[0]: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1.

KeyboardInterrupt: 

In [ ]:
# save model
dqn._model.save(f'models/model1')

2022-01-22 16:37:14.665630: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model1/assets


INFO:tensorflow:Assets written to: models/model1/assets
